In [4]:
# Welcome to your new notebook
df_product_bronze = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_bronze_lakehouse.Lakehouse/Tables/dbo.product")

StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 6, Finished, Available, Finished)

In [23]:
display(df_product_bronze)

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 25, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 933fb2ea-ffea-4eec-8ef9-7f3f0f6b844b)

In [24]:
df_product_bronze.printSchema()

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 26, Finished, Available, Finished)

root
 |-- ProductKey: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Standard_Cost: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Subcategory: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Background_Color_Format: string (nullable = true)
 |-- Font_Color_Format: string (nullable = true)



In [5]:
df_sales_bronze = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_bronze_lakehouse.Lakehouse/Tables/dbo.sales")

StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 7, Finished, Available, Finished)

In [45]:
from pyspark.sql.functions import col, sum

df_sales_bronze.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in df_sales_bronze.columns
]).show()



StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 47, Finished, Available, Finished)

+----------------+---------+----------+-----------------+-------------+---------+-----------+----------------+
|SalesOrderNumber|OrderDate|ProductKey|SalesTerritoryKey|OrderQuantity|UnitPrice|SalesAmount|TotalProductCost|
+----------------+---------+----------+-----------------+-------------+---------+-----------+----------------+
|               0|        0|         0|                0|            0|        0|          0|               0|
+----------------+---------+----------+-----------------+-------------+---------+-----------+----------------+



In [26]:
display(df_sales_bronze)

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 96524d12-2b30-45cb-b772-3ec8456be47f)

In [27]:
df_sales_bronze.printSchema()

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 29, Finished, Available, Finished)

root
 |-- SalesOrderNumber: string (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- ProductKey: string (nullable = true)
 |-- SalesTerritoryKey: string (nullable = true)
 |-- OrderQuantity: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- SalesAmount: string (nullable = true)
 |-- TotalProductCost: string (nullable = true)



In [3]:
df_region_bronze = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_bronze_lakehouse.Lakehouse/Tables/dbo.region")

StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 5, Finished, Available, Finished)

In [29]:
display(df_region_bronze)

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 312a89d8-dddb-4eb7-9d10-3017bea00222)

In [30]:
df_region_bronze.printSchema()

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 32, Finished, Available, Finished)

root
 |-- SalesTerritoryKey: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Group: string (nullable = true)



In [46]:
from pyspark.sql.functions import col, sum

df_region_bronze.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in df_region_bronze.columns
]).show()

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 48, Finished, Available, Finished)

+-----------------+------+-------+-----+
|SalesTerritoryKey|Region|Country|Group|
+-----------------+------+-------+-----+
|                0|     0|      0|    0|
+-----------------+------+-------+-----+



In [47]:
from pyspark.sql.functions import col, sum

df_product_bronze.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in df_product_bronze.columns
]).show()

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 49, Finished, Available, Finished)

+----------+-------+-------------+-----+-----------+--------+-----------------------+-----------------+
|ProductKey|Product|Standard_Cost|Color|Subcategory|Category|Background_Color_Format|Font_Color_Format|
+----------+-------+-------------+-----+-----------+--------+-----------------------+-----------------+
|         0|      0|            0|    0|          0|       0|                      0|                0|
+----------+-------+-------------+-----+-----------+--------+-----------------------+-----------------+



In [31]:
from datetime import datetime
import uuid
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType

def log_data_quality_issues_dtypes(df, expected_schema: dict, log_file_path: str, source_table: str, target_table: str):
    """
    Logs data type mismatches before transformation to a JSON file (like Bronze logs).

    Parameters:
    - df: Input DataFrame from Bronze.
    - expected_schema: Dict with expected column types. e.g., {'ProductKey': 'int'}
    - log_file_path: Path to save JSON logs in Files (not Tables).
    - source_table: Table name in Bronze.
    - target_table: Table name in Silver.
    """

    logs = []
    for col_name, expected_type in expected_schema.items():
        actual_type = dict(df.dtypes).get(col_name)
        if actual_type != expected_type:
            logs.append(Row(
                log_id=str(uuid.uuid4()),
                column=col_name,
                actual_type=actual_type,
                expected_type=expected_type,
                status="Mismatch",
                timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                source_table=source_table,
                target_table=target_table
            ))

    if logs:
        log_schema = StructType([
            StructField("log_id", StringType(), True),
            StructField("column", StringType(), True),
            StructField("actual_type", StringType(), True),
            StructField("expected_type", StringType(), True),
            StructField("status", StringType(), True),
            StructField("timestamp", StringType(), True),
            StructField("source_table", StringType(), True),
            StructField("target_table", StringType(), True)
        ])

        log_df = spark.createDataFrame(logs, schema=log_schema)

        try:
            # Save logs in JSON format (like Bronze), in the FILES section
            log_df.coalesce(1).write.mode("append").json(log_file_path)
            print(f"Data quality issues logged to JSON at: {log_file_path}")
        except Exception as e:
            print(f"Failed to write log: {str(e)}")
    else:
        print("No mismatches found.")


StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 33, Finished, Available, Finished)

In [32]:
expected_schema_region = {
    "SalesTerritoryKey": "int",
    "Region": "string",
    "Country": "string",
    "Group": "string"
}
expected_schema_sales = {
    "SalesOrderNumber": "string",
    "OrderDate": "date",
    "ProductKey": "int",
    "SalesTerritoryKey": "int",
    "OrderQuantity": "int",
    "UnitPrice": "double",
    "SalesAmount": "double",
    "TotalProductCost": "double"
}

expected_schema_product = {
    "ProductKey": "int",
    "Product": "string",
    "Standard_Cost": "double",
    "Color": "string",
    "Subcategory": "string",
    "Category": "string",
    "Background_Color_Format": "string",
    "Font_Color_Format": "string"
}




StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 34, Finished, Available, Finished)

In [33]:
lh_tables_path='abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Tables/dbo.log'
log_data_quality_issues_dtypes(df_product_bronze,expected_schema_product,lh_tables_path,'dbo.bronze_product','dbo.silver')

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 35, Finished, Available, Finished)

Data quality issues logged to JSON at: abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Tables/dbo.log


In [34]:
lh_tables_path='abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Tables/dbo.log'
log_data_quality_issues_dtypes(df_sales_bronze,expected_schema_sales,lh_tables_path,'dbo.bronze_sales','dbo.silver')

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 36, Finished, Available, Finished)

Data quality issues logged to JSON at: abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Tables/dbo.log


In [35]:
lh_tables_path='abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Tables/dbo.log'
log_data_quality_issues_dtypes(df_region_bronze,expected_schema_region,lh_tables_path,'dbo.bronze_region','dbo.silver')

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 37, Finished, Available, Finished)

Data quality issues logged to JSON at: abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Tables/dbo.log


In [37]:
display(df)

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 39, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 39adc843-8ea5-4ae7-b3f3-1f57825dbfe4)

null values DQC

In [38]:
from datetime import datetime
import uuid
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType

def log_null_checks(df, log_file_path: str, source_table: str, target_table: str):
    """
    Logs columns that contain null values in the DataFrame to JSON file.

    Parameters:
    - df: Input DataFrame to check.
    - log_file_path: Path to save JSON logs in Files section.
    - source_table: Table name in Bronze.
    - target_table: Table name in Silver.
    """

    logs = []
    null_counts = df.select([df[col].isNull().cast("int").alias(col) for col in df.columns]) \
                    .groupBy().sum().collect()[0].asDict()

    for col_name, null_count in null_counts.items():
        if null_count > 0:
            logs.append(Row(
                log_id=str(uuid.uuid4()),
                column=col_name,
                null_count=str(null_count),
                status="Nulls Found",
                timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                source_table=source_table,
                target_table=target_table
            ))

    if logs:
        log_schema = StructType([
            StructField("log_id", StringType(), True),
            StructField("column", StringType(), True),
            StructField("null_count", StringType(), True),
            StructField("status", StringType(), True),
            StructField("timestamp", StringType(), True),
            StructField("source_table", StringType(), True),
            StructField("target_table", StringType(), True)
        ])

        log_df = spark.createDataFrame(logs, schema=log_schema)

        try:
            log_df.coalesce(1).write.mode("append").json(log_file_path)
            print(f"Null value logs written to: {log_file_path}")
        except Exception as e:
            print(f"Failed to write null check log: {str(e)}")
    else:
        print("No nulls found in any column.")


StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 40, Finished, Available, Finished)

In [39]:
from datetime import datetime
import uuid
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType

def log_null_checks(df, log_file_path: str, source_table: str, target_table: str):
    """
    Logs columns that contain null values in the DataFrame to JSON file.

    Parameters:
    - df: Input DataFrame to check.
    - log_file_path: Path to save JSON logs in Files section.
    - source_table: Table name in Bronze.
    - target_table: Table name in Silver.
    """

    logs = []
    null_counts = df.select([df[col].isNull().cast("int").alias(col) for col in df.columns]) \
                    .groupBy().sum().collect()[0].asDict()

    for col_name, null_count in null_counts.items():
        if null_count > 0:
            logs.append(Row(
                log_id=str(uuid.uuid4()),
                column=col_name,
                null_count=str(null_count),
                status="Nulls Found",
                timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                source_table=source_table,
                target_table=target_table
            ))

    if logs:
        log_schema = StructType([
            StructField("log_id", StringType(), True),
            StructField("column", StringType(), True),
            StructField("null_count", StringType(), True),
            StructField("status", StringType(), True),
            StructField("timestamp", StringType(), True),
            StructField("source_table", StringType(), True),
            StructField("target_table", StringType(), True)
        ])

        log_df = spark.createDataFrame(logs, schema=log_schema)

        try:
            log_df.coalesce(1).write.mode("append").json(log_file_path)
            print(f"Null value logs written to: {log_file_path}")
        except Exception as e:
            print(f"Failed to write null check log: {str(e)}")
    else:
        print("No nulls found in any column.")


StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 41, Finished, Available, Finished)

In [40]:
null_log_file_path='abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Files//data_quality_null_logs/'
log_null_checks(df_product_bronze,null_log_file_path,'dbo.bronze_product','silver-files_product')

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 42, Finished, Available, Finished)

No nulls found in any column.


In [41]:
null_log_file_path='abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Files//data_quality_null_logs/'
log_null_checks(df_sales_bronze,null_log_file_path,'dbo.bronze_sales','silver-files_sales')

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 43, Finished, Available, Finished)

No nulls found in any column.


In [42]:
null_log_file_path='abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Files//data_quality_null_logs/'
log_null_checks(df_region_bronze,null_log_file_path,'dbo.bronze_region','silver-files_region')

StatementMeta(, 7c83cfaa-36bd-4b41-8fe7-2b0b52328e88, 44, Finished, Available, Finished)

No nulls found in any column.


In [1]:

from datetime import datetime
import uuid
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType

def log_duplicate_checks(df, log_file_path: str, source_table: str, target_table: str, subset_cols: list = None):
    """
    Logs duplicate rows based on specified columns to JSON.

    Parameters:
    - df: Input DataFrame to check.
    - log_file_path: Path to save JSON logs in Files section.
    - source_table: Table name in Bronze.
    - target_table: Table name in Silver.
    - subset_cols: Optional list of columns to check duplicates on. If None, all columns are used.
    """
    if subset_cols is None:
        subset_cols = df.columns

    # Detect duplicates
    dup_df = df.groupBy(subset_cols).count().filter("count > 1")

    logs = []
    dup_count = dup_df.count()

    if dup_count > 0:
        logs.append(Row(
            log_id=str(uuid.uuid4()),
            duplicate_count=str(dup_count),
            duplicate_on_columns=",".join(subset_cols),
            status="Duplicates Found",
            timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            source_table=source_table,
            target_table=target_table
        ))

    if logs:
        log_schema = StructType([
            StructField("log_id", StringType(), True),
            StructField("duplicate_count", StringType(), True),
            StructField("duplicate_on_columns", StringType(), True),
            StructField("status", StringType(), True),
            StructField("timestamp", StringType(), True),
            StructField("source_table", StringType(), True),
            StructField("target_table", StringType(), True)
        ])

        log_df = spark.createDataFrame(logs, schema=log_schema)

        try:
            log_df.coalesce(1).write.mode("append").json(log_file_path)
            print(f"Duplicate logs written to: {log_file_path}")
        except Exception as e:
            print(f"Failed to write duplicate check log: {str(e)}")
    else:
        print("No duplicates found.")



StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 3, Finished, Available, Finished)

In [6]:
file_path='abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Files/dulicates_logs/'
log_duplicate_checks(df_product_bronze,file_path,'bronze_product','silver_files_product')

StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 8, Finished, Available, Finished)

No duplicates found.


In [7]:
file_path='abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Files/dulicates_logs/'
log_duplicate_checks(df_product_bronze,file_path,'bronze_product','silver_files_product',['ProductKey'])

StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 9, Finished, Available, Finished)

No duplicates found.


In [8]:
file_path='abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Files/dulicates_logs/'
log_duplicate_checks(df_sales_bronze,file_path,'bronze_sales','silver_files_sales')

StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 10, Finished, Available, Finished)

No duplicates found.


In [10]:
file_path='abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Files/dulicates_logs/'
log_duplicate_checks(df_sales_bronze,file_path,'bronze_sales','silver_files_sales',['SalesOrderNumber','ProductKey'])

StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 12, Finished, Available, Finished)

No duplicates found.


In [11]:
file_path='abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Files/dulicates_logs/'
log_duplicate_checks(df_region_bronze,file_path,'bronze_region','silver_files_region')

StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 13, Finished, Available, Finished)

No duplicates found.


TRANSFORMATIONS

In [12]:
display(df_product_bronze)

StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 80d0e63f-244e-4bee-944e-bd1d57a5b6af)

In [13]:
from datetime import datetime
import uuid
from pyspark.sql import Row
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import StructType, StructField, StringType

def log_transformation(log_file_path: str, column_name: str, change_desc: str, source_table: str, target_table: str):
    """
    Logs transformations applied to columns.

    Parameters:
    - log_file_path: Path to save JSON logs.
    - column_name: Name of the transformed column.
    - change_desc: Description of the transformation.
    - source_table: Table name in Bronze.
    - target_table: Table name in Silver.
    """
    logs = [Row(
        log_id=str(uuid.uuid4()),
        column=column_name,
        change_description=change_desc,
        timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        source_table=source_table,
        target_table=target_table
    )]

    log_schema = StructType([
        StructField("log_id", StringType(), True),
        StructField("column", StringType(), True),
        StructField("change_description", StringType(), True),
        StructField("timestamp", StringType(), True),
        StructField("source_table", StringType(), True),
        StructField("target_table", StringType(), True)
    ])

    log_df = spark.createDataFrame(logs, schema=log_schema)

    try:
        log_df.coalesce(1).write.mode("append").json(log_file_path)
        print(f"Transformation log written to: {log_file_path}")
    except Exception as e:
        print(f"Failed to write transformation log: {str(e)}")






StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 15, Finished, Available, Finished)

In [14]:
df_product_bronze = df_product_bronze.withColumn("Standard_Cost", regexp_replace(col("Standard_Cost"), "[$,]", "").cast("double"))

trans_log_file_path = "abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Files/transformations_log/"
log_transformation(
    log_file_path=trans_log_file_path,
    column_name="Standard_Cost",
    change_desc="Removed '$' symbol and converted to double",
    source_table="dbo.bronze_product",
    target_table="silver-files_product"
)  

StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 16, Finished, Available, Finished)

Transformation log written to: abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Files/transformations_log/


In [15]:
display(df_product_bronze)

StatementMeta(, d9624a6d-e43b-4c8b-9799-d9db8765b8ea, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2af21de8-7ee3-42b6-bdef-f2937f7c7a7f)